In [1]:
import openai

In [2]:
(openai.api_key,) = !cat ../data/auth/openai.api.key

In [3]:
functions = [
    {
        "name": "get_label",
        "description": "判定ラベル. BLEACHは0, コミックLOは1",
        "parameters": {
            "type": "object",
            "properties": {
                "label": {
                    "type": "integer",
                    "enum": [0, 1],
                    "description": "判定ラベル. BLEACHは0, コミックLOは1",
                },
            },
            "required": ["label"],
        },
    }
]

In [5]:
# 少し3.5-turboで練習して、ある程度精度が出てきたら4に移行する
import json


def get_gpt_pred(poem):
    response = openai.ChatCompletion.create(
        # model="gpt-3.5-turbo",
        model="gpt-4",
        messages=[
            {
                "role": "user",
                "content": f"""
あなたにポエムをBLEACHのものかCOMIC LOのものか判定してもらいます.
BLEACHの例は以下の通りです。
- 伏して生きるな、立ちて死すべし
- そう、何ものもわたしの世界を変えられはしない
- 僕は、ついてゆけるだろうか君のいない世界のスピードに
- 剣を握らなければおまえを守れない剣を握ったままではおまえを抱き締められない
- 軋む軋む浄罪の塔光のごとくに世界を貫く揺れる揺れる背骨の塔堕ちてゆくのはぼくらか空か

COMIC LOの例は以下の通りです。
- 女って、重いんだな。
- 前を向く君は、青空より美しい。
- 春の子供。
- ロリコンに春は来ない。
- この娘の母親が本当に好きだった。それを今、思い出した。
- 僕は望む。そして叶うことを決して望まない。

それでは、以下のポエムをBLEACHのものかCOMIC LOのものか判定してください.
"{poem}"
BLEACHの場合は0を、COMIC LOの場合は1を出力してください.
    """,
            },
        ],
        functions=functions,
        function_call="auto",
        max_tokens=50,
    )

    # print(response.choices[0]["message"])
    return int(
        json.loads(response.choices[0]["message"]["function_call"]["arguments"])[
            "label"
        ]
    )

get_gpt_pred("今度の終末はどこへ行く？")

0

In [6]:
import pandas as pd

test_df = pd.read_csv("../data/test.tsv", sep="\t")

In [12]:
# poem2predがなければ新しく作る
try:
    poem2pred
except Exception as e:
    poem2pred = {}

In [13]:
for poem in test_df["poem"]:
    if poem not in poem2pred.keys():
        print(len(poem2pred))
        poem2pred[poem] = get_gpt_pred(poem)

test_df["pred"] = test_df["poem"].map(lambda x: poem2pred[x])
test_df.to_csv("../data/test_w_gpt_20231203.tsv", sep="\t", index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [14]:
from sklearn.metrics import accuracy_score

accuracy_score(test_df["label"], test_df["pred"])

0.8148148148148148